In [1]:
import sys
import re
from enum import Enum
from dataclasses import dataclass
from typing import List

In [ ]:
class CellType(Enum):
    EMPTY = 0
    BLOCK = 1
    SOURCE = 2
    TARGET = 3
    VIA = 4

@dataclass
class Cell:
    kind: CellType
    cost: int = 0

@dataclass
class Coord:
    row: int
    col: int
    layer: int


class MazeRouter:
    NUM_LAYERS = 2

    def __init__(self,
                 in_path: str,
                 out_path: str,  #Check if needed later on
                 routing: str = "routing.txt",
                 dimensions: str = "dimensions.txt"):
        self.in_path = in_path
        self.out_path = out_path
        self.routing = routing
        self.dimensions = dimensions

        self.cols = 0
        self.rows = 0
        self.Direction = 0
        self.via = 0

        self.grid: List[List[List[Cell]]] = []
        self.nets: List[List[Coord]] = []

        self.clear_routings()
        self.load_config()
        self.save_dimensions()

    def clear_routings(self):
        with open(self.routing, 'w'):
            pass
        print("clear_routing Successful")

    def load_config(self):
        with open(self.in_path, 'r') as file:
            header = file.readline().strip()
            c, r, b, v = map(int, header.split(','))
            self.cols, self.rows = c, r
            self.Direction, self.via = b, v

            self.grid = []
            for k in range(self.NUM_LAYERS):
                layer = []
                for j in range(self.rows):
                    row = []
                    for i in range(self.cols):
                        row.append(Cell(CellType.EMPTY))
                    layer.append(row)
                self.grid.append(layer)

            for line in file:
                line = line.strip()
                if line.startswith("OBS"):
                    self.add_obstacle(line)
                elif line.startswith("net"):
                    self.add_net(line)

    def save_dimensions(self):
        with open(self.dimensions, 'w') as file:
            file.write(f"{self.cols} {self.rows}\n")


    def add_obstacle(self, text: str):
        m = re.match(r"OBS\s*\((\d+),\s*(\d+),\s*(\d+)\)", text)
        if not m:
            print("Incorrect Format.", file=sys.stderr)
            return
        layer, col, row = map(int, m.groups())
        if 0 <= layer < self.NUM_LAYERS and 0 <= row < self.rows and 0 <= col < self.cols:
            self.grid[layer][row][col].kind = CellType.BLOCK
        else:
            print("Obstacle out of Range", file=sys.stderr)

    def add_net(self, text: str):
        path = []
        for m in re.finditer(r"\((\d+),\s*(\d+),\s*(\d+)\)", text):
            layer, col, row = map(int, m.groups())
            if 0 <= layer < self.NUM_LAYERS and 0 <= row < self.rows and 0 <= col < self.cols:
                path.append(Coord(row, col, layer))
            else:
                print("Coordinates out of Range", file=sys.stderr)
                return
        self.nets.append(path)

In [11]:
def main():
    input_file = input("Enter input file name: ").strip()
    output_file = input("Enter output file name: ").strip()

    router = MazeRouter(input_file, output_file)

    print("\nRouter initialized")
    print(f"Maze dimensions: {router.rows} x {router.cols}")
    print(f"Direction penalty: {router.Direction}")
    print(f"Via penalty: {router.via}")

if __name__ == "__main__":
    main()


clear_routing Successful

Router initialized
Maze dimensions: 8 x 8
Direction penalty: 5
Via penalty: 10
